# **프로토타입 모델 구조 개요**
- 데이터: 전처리 작업 완료된 내용으로 구성(구조화된 형태로 추출 -> 쳥킹)
- 임베딩 모델: OPENAI의 임베딩 모델 사용
- 벡터DB: 오픈소스 기반의 FAISS DB 사용, 우선 테스트를 위해 가장 복잡한 문서구조를 지닌 '수강편람'만 적재
- Retriever(검색기): Dense Retriever(벡터DB기반의 검색기) 단일, 반환 문서 5개, 유사도 기반 검색
- 프롬프트 설계
    - 시스템프롬프트: 한국외대 학사정보 챗봇 인지
    - 컨텍스트 기반 답변 지시
    -  +) 반존대, 반말 사용한 응답이 다수 출현 -> 높임말 사용 지시
- LLM 모델: GPT 3.5 Turbo
- 체인 구성: 질의를 받아 답변을 생성하기까지의 구조를 정의
    - 질의내용, 검색 문서를 기존에 정의한 시스템 프롬프트 형태로 LLM에 전달
    - LLM이 자연어로 응답 생성
    - outputpaser를 이용해 출력

In [2]:
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import json
import pandas as pd
from dotenv import load_dotenv
from collections import defaultdict

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# ✅ 1. 강의시간표 문서 로드 함수
def load_subject_by_area(filepath: str, filename: str) -> list[Document]:
    df = pd.read_csv(filepath)
    documents = []
    grouped = df.groupby("개설영역")
    for area, group_df in grouped:
        contents = []
        for _, row in group_df.iterrows():
            if pd.notna(row["학년"]):
                year_text = f"{int(float(row['학년']))}학년 대상"
            else:
                year_text = "전체 학년 대상"
            row_text = (
                f"[{row['교과목명']}] {year_text} / 교수: {row['담당교수']} / "
                f"강의실: {row['강의시간/강의실']} / 학점: {row['학점']} / 계획서: {row['강의계획서']}"
            )
            contents.append(row_text)
        page_content = "\n\n".join(contents)
        metadata = {
            "filename": filename,
            "개설영역": area,
            "doc_type": "강의시간표"
        }
        documents.append(Document(page_content=page_content.strip(), metadata=metadata))
    return documents

def load_all_subject_documents(directory: str) -> list[Document]:
    all_documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            documents = load_subject_by_area(filepath, filename)
            all_documents.extend(documents)
    return all_documents


# 전공 가이드북 PDF 로드 + 청킹
def load_major_guidebook(pdf_path: str) -> list[Document]:
    """
    전공 가이드북 PDF를 로드하여 페이지별로 청킹한 Document 리스트 반환
    - 페이지별 Document를 청킹하여 세부 정보 유지
    - metadata에는 source 경로만 포함
    """
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()  # 페이지별 Document 리스트

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200,
        chunk_overlap=200,
        separators=["\n\n", "\n", ".", " "],
    )
    chunks = splitter.split_documents(pages)
    return chunks

# 전공 가이드북 PDF 로드 + 청킹
def load_major_guidebook(pdf_path: str) -> list[Document]:
    """
    전공 가이드북 PDF를 로드하여 페이지별로 청킹한 Document 리스트 반환
    - 페이지별 Document를 청킹하여 세부 정보 유지
    - metadata에는 source 경로만 포함
    """
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()  # 페이지별 Document 리스트

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200,
        chunk_overlap=200,
        separators=["\n\n", "\n", ".", " "],
    )
    chunks = splitter.split_documents(pages)
    return chunks


#  수강편람 JSON 로드 → 페이지별 Document 분리 → 청킹 적용
def load_sugang_pram(json_path: str, doc_type: str = "수강편람") -> list[Document]:
    with open(json_path, "r", encoding="utf-8") as f:
        result = json.load(f)

    elements = result["elements"]
    page_chunks = defaultdict(list)

    for el in elements:
        page = el.get("page", 0)
        content = el.get("content", {}).get("markdown", "").strip()
        if content:
            page_chunks[page].append((el["category"], content, el))

    docs = []
    for page, items in page_chunks.items():
        combined_text = "\n\n".join(f"## [{cat}] ##\n{txt}" for cat, txt, _ in items)
        metadata = {
            "doc_type": doc_type,
            "page": page,
            "num_elements": len(items),
            "element_ids": [el["id"] for _, _, el in items],
            "categories": list(set(cat for cat, _, _ in items)),
        }
        docs.append(Document(page_content=combined_text, metadata=metadata))

    # 청킹
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", ".", " "],
    )
    chunks = splitter.split_documents(docs)
    return chunks

In [3]:
# 2. 문서 로딩
timetable_path = "data/Timetable_Crawling_Data"
guidebook_path = "data/major_guide_2025.pdf"
pram_path = "data/pram_2025_1.json"

subject_docs = load_all_subject_documents(timetable_path)
guidebook_docs = load_major_guidebook(guidebook_path)
pram_docs = load_sugang_pram(pram_path)

# 3.임베딩 
embedding = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=OPENAI_API_KEY)

# FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(pram_docs, embedding=embedding)

# **기본 검색기 + 템플릿 조합**

In [4]:
# 검색기
retriever = vectorstore.as_retriever(
    search_type="similarity",      # 질문과 컨텍스트의 유사성을 기준으로 검색
    search_kwargs={
        "k": 5                     # 검색할 문서 수
    }
)

# ✅ 4. 프롬프트 + 체인 구성
prompt = PromptTemplate.from_template("""
너는 한국외대 학사 관련 질문에 답변하는 챗봇이야.
다음은 검색된 문서 내용이야. 이 내용을 바탕으로 사용자의 질문에 답변해.
답을 모르면 모른다고 해. 무조건 한국어로 답변해.

#질문:
{question}

#문서 내용:
{context}

#답변:
""")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key = OPENAI_API_KEY)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
# 1번 질문: 쉽고 명확한 질문
question = "전과신청 시행 시기 알려줘" # 명확한 질문
response = chain.invoke(question)
print(response)

전과신청은 매년 4월 중에 시행됩니다.


**전과 내용 컨텍스트 기반해 답변함**

In [10]:
docs = vectorstore.similarity_search(question, k=5)
for doc in docs:
    print(doc.metadata)          # 문서 메타정보 (예: source, page 등)
    print(doc.page_content)  # 문서 내용 미리보기

{'doc_type': '수강편람', 'page': 6, 'num_elements': 9, 'element_ids': [16, 17, 18, 19, 20, 21, 22, 23, 24], 'categories': ['list', 'footer', 'paragraph', 'heading1', 'table']}
## [paragraph] ##
나. 수강신청 (※ 수강신청 전 본인 종합정보시스템의 비밀번호 변경 권장)

## [paragraph] ##
1) 학교 홈페이지 수강신청 임시화면(별도 기간) ‘바로가기’ 혹은 팝업 ‘수강신청 바로가기’
2) 수강신청은 수강신청기간 중 반드시 학생 본인이 하여야 함

## [table] ##
| 신청학년 (2025-1학기 등록횟수 기준) ※ 편입생은 학년 기준 | 신청학년 (2025-1학기 등록횟수 기준) ※ 편입생은 학년 기준 | 일정 | 비고 |
| --- | --- | --- | --- |
| 재 학 생 | 4학년 (등록 7회 이상) | 2. 3.(월) | - 학교 홈페이지 : www.hufs.ac.kr - ‘수강신청/예비수강신청함 바로가기’ - 이용시간 : 10:00 ~ 15:00 |
| 재 학 생 | 3학년 (등록 5~6회) | 2. 4.(화) | - 학교 홈페이지 : www.hufs.ac.kr - ‘수강신청/예비수강신청함 바로가기’ - 이용시간 : 10:00 ~ 15:00 |
| 재 학 생 | 2학년 (등록 3~4회) | 2. 5.(수) | - 학교 홈페이지 : www.hufs.ac.kr - ‘수강신청/예비수강신청함 바로가기’ - 이용시간 : 10:00 ~ 15:00 |
| 재 학 생 | 1학년 (등록 1~2회) | 2. 6.(목) | - 학교 홈페이지 : www.hufs.ac.kr - ‘수강신청/예비수강신청함 바로가기’ - 이용시간 : 10:00 ~ 15:00 |
| 재 학 생 | 전체 학년 | 2. 7.(금) | - 학교 홈페이지 : www.hufs.ac.kr - ‘수강신청/예비수강신청함 바로가기’ - 이용시간 

In [11]:
# 2번 질문: 컨텍스트 참고만으로 답변하기 어려울 수 있는 질문
question = "감기로 병원다녀와서 진료확인서 제출하면 유고결석 인정 가능해?" # 해석난이도 높은 질문
response = chain.invoke(question)
print(response)

병원에서 발급받은 진료확인서를 제출하면 유고결석이 인정될 수 있어.


- **수강편람에서 '감기', '진료확인서'로 유고결석이 인정될 수 있는지 여부는 확인 불가**
- **즉 LLM이 컨텍스트를 어떻게 추론하고 해석하는지에 따라 달리는데, 이 경우 예상되는 우수한 답변은 '정확한 내용은 ~를 참고해라'라는 식으로 제안**
- **개선 방안: few-shot prompting**

![image.png](test_images/감기쿼리_컨텍스트.png)

In [7]:
# 3번 질문: 추상적인 질문
question = "후기 이중에 대해 설명해줘" # 축약어 사용
response = chain.invoke(question)
print(response)

후기이중전공은 학기 3/4선 이후부터 다음 학기 1/4선 이전까지만 취소 및 포기가 가능하며, 졸업사정회의를 전후로 당학기 또는 다음 학기에 졸업할 수 있다고 해.


- **다소 추상적인 질문에 대해서 원하는 응답을 가져오지 못함**
- **잘못된 쳥크를 가져옴**
- **'후기 이중'에 대한 컨텍스트를 가져오긴 했지만, 해당 페이지의 쳥크를 모두 가져왔어야 함**
- **즉 질문의 의도를 명확히 파악하지 못한 점, 질문과 유사한 쳥크를 모두 가져오지 못한 점** 
- **개선방안: Parent-Document Retriever, MultiQueryRetriever**

![image](test_images/후기이중쿼리_컨텍스트.png)

In [24]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 정의
prompt = PromptTemplate.from_template("""
너는 한국외대 학사 관련 질문에 답변하는 챗봇이야.
다음은 검색된 문서 내용이야. 이 내용을 바탕으로 사용자의 질문에 답변해.
답을 모르면 모른다고 해. 무조건 한국어로 높임말을 사용해서 답변해.

#질문:
{question}

#문서 내용:
{context}

#답변:
""")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=OPENAI_API_KEY)

qa_chain = (
    prompt
    | llm
    | StrOutputParser()
)

# 문서 검색 및 context 생성
query = "후기 이중에 대해 설명해줘"
retrieved_docs = retriever.invoke(query)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

# 체인 실행
result = qa_chain.invoke({"question": query, "context": context})

# 출력
print("💬 GPT 응답:\n", result)


💬 GPT 응답:
 후기이중전공은 학기 3/4선 이후부터 다음 학기 1/4선 이전까지만 취소 및 포기가 가능하며, 졸업사정회의를 전후로 당학기 또는 다음 학기에 졸업할 수 있습니다. 문의는 학사종합지원센터(Tel : 031-330-4026)로 하시면 됩니다.


In [26]:
print("\n📚 참고된 문서:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n[{i}] {doc.metadata}")
    print(doc.page_content[:300], "...")


📚 참고된 문서:

[1] {'doc_type': '수강편람', 'page': 75, 'num_elements': 12, 'element_ids': [611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622], 'categories': ['heading1', 'paragraph', 'footer', 'list', 'table']}
## [heading1] ##
# 마. 신청시기 : 매년 4월 중 (연1회)

## [paragraph] ##
바. 후기이중전공 취소 및 포기 : 학기 3/4선 이후부터 다음 학기 1/4선 이전까지만
가능하며, 졸업사정회의를 전후로 당학기 또는 다음 학기에 졸업할 수 있음

## [list] ##
- 사. 문의 : 학사종합지원센터(Tel : 031-330-4026)

## [footer] ##
- 75 - ...

[2] {'doc_type': '수강편람', 'page': 3, 'num_elements': 4, 'element_ids': [4, 5, 6, 7], 'categories': ['table', 'heading1', 'paragraph']}
## [paragraph] ##
※ 위 학사일정은 학사운영상 변경될 수 있습니다. ...

[3] {'doc_type': '수강편람', 'page': 66, 'num_elements': 7, 'element_ids': [546, 547, 548, 549, 550, 551, 552], 'categories': ['heading1', 'paragraph', 'footer', 'list', 'table']}
## [paragraph] ##
7) 문의처 : 철학과 사무실(Tel : 031-330-4282)]

## [footer] ##
- 66 - ...

[4] {'doc_type': '수강편람', 'page': 90, 'num_elements': 16, 'element_ids': [775, 776, 777, 778, 779, 780, 781,

In [74]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 정의
prompt = PromptTemplate.from_template("""
너는 한국외대 학사 관련 질문에 답변하는 챗봇이야.
다음은 검색된 문서 내용이야. 이 내용을 바탕으로 사용자의 질문에 답변해.
답을 모르면 모른다고 해. 무조건 한국어로 높임말을 사용해서 답변해.

#질문:
{question}

#문서 내용:
{context}

#답변:
""")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=OPENAI_API_KEY)

qa_chain = (
    prompt
    | llm
    | StrOutputParser()
)

# 문서 검색 및 context 생성
query = "bme 졸업학점 알려줘"
retrieved_docs = retriever.invoke(query)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

# 체인 실행
result = qa_chain.invoke({"question": query, "context": context})

# 출력
print("💬 GPT 응답:\n", result)


💬 GPT 응답:
 졸업학점은 전공심화(단일전공)으로 134학점이 필요합니다.


- **부정확한 답변**
- **축약어 자체에는 생각보다 잘 대응한 모습**
- **다만, 잘못된 컨텍스트를 가져옴**
- **즉 정보검색의 정밀도가 떨어져서 발생한 문제**
- **개선방안: 프롬프트 엔지니어링, MultiQueryRetriever, reranker, BM25**

![image](test_images/bme쿼리_컨텍스트.png)

In [75]:
print("\n📚 참고된 문서:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n[{i}] {doc.metadata}")
    print(doc.page_content[:300], "...")


📚 참고된 문서:

[1] {'doc_type': '수강편람', 'page': 56, 'num_elements': 11, 'element_ids': [442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452], 'categories': ['table', 'list', 'paragraph', 'footer']}
## [paragraph] ##
4) 학생 선발

## [paragraph] ##
- 바이오데이터사이언스 융합전공은 모든 전공의 학생들이 선택할 수 있음. 각 전공의
학생들이 이 융합전공을 이수하는 방식은 다음과 같음

## [paragraph] ##
<바이오데이터사이언스> 전공이수 방식

## [table] ##
| 학생 구분 | 수강 과목 | 기대 효과 |
| --- | --- | --- |
| SW 전공학생 (컴퓨터공학부, 정보통신공학과) | SW 전공 57학점 SW 영역 추가 21학점 바이오메디컬영역 21학점 | - SW 영 ...

[2] {'producer': 'Adobe PDF Library 16.0', 'creator': 'Adobe InDesign 16.4 (Macintosh)', 'creationdate': '2024-06-21T10:55:17+09:00', 'moddate': '2024-06-21T11:00:00+09:00', 'trapped': '/False', 'source': 'data/major_guide_2025.pdf', 'total_pages': 37, 'page': 33, 'page_label': '34'}
63 62글로벌캠퍼스 GLOBAL CAMPUSDEPARTMENT  OF
BIOMEDICAL ENGINEERING
바이오메디컬공학부
바이오메디컬공학부  63Department of Biomedical Engineering바이오메디컬공학부전화번호 031-330-4723
홈페이지  bme.hufs.ac.kr
전공소개 
한국외국어대학교 바이오메디컬공학부는 글로벌 바이오산업의 핵심 
인

# **Retriever 적용**
- 가장 유사한 30개 문서를 가져오는 기본 검색기 이용
- 30개 문서를 가져온 다음 → CrossEncoder 모델로 다시 평가해서 상위 15개만 남김 -> 의미 중복이나 불필요한 정보 제거
- 히스토리 인식 Retriever ->  이전 대화 문맥 반영해 답변 생성
- chat history를 이용해 사용자가 이전에 했던 질문과 답변들을 기억해서 다음 질문에 활용할 수 있도록함
- RunnableWithMessageHistory를 이용해 history포함한채 RAG체인 실행

In [8]:
import os, time
from dotenv import load_dotenv
from uuid import uuid4
from operator import itemgetter

from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.document_transformers import LongContextReorder
from langchain.chains import create_history_aware_retriever
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.chains import create_retrieval_chain

In [9]:
# 1. 환경 변수 로드
load_dotenv()
#PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 2. 사용자 질문 전처리
def normalize_query(query: str) -> str:
    abbreviation_map = {
        "글스산": "글로벌스포츠산업전공",
        "바메공": "바이오메디컬공학전공",
        "BME": "바이오메디컬공학전공",
        "이중": "이중전공",
        "통대": "통번역대학",
        "공대": "공과대학",
        "일통": "일본어통번역학과",
        "영통": "영어통번역학과",
        "독통": "독일어통번역학과",
        "스통": "스페인어통번역학과",
        "마인어":"말레이시아인도네시아통번역학과",
        "GBT": "Global Business&Technology",
        "융인": "융합인재학과",
        "중통": "중국어통번역학과",
        "국금": "국제금융학과",
        "이통": "이탈리아어통번역학과",
        "태통": "태국어통번역학과",
        "정통": "정보통신공학과",
        "산공": "산업경영공학과",
        "산경공": "산업경영공학과",
        "파에": "Finance&AI융합학부",
        "데융": "AI데이터융합학부",
        "글자전": "글로벌자유전공학부",
        "자전": "글로벌자유전공학부",
        "대영": "대학영어",
        "데사":"데이터사이언스",
        "국리":"국가리더전공",
        "세크": "세르비아·크로아티아",
        "그불": "그리스·불가리아",
        "전물": "전자물리학과",
        "생공": "생명공학과",
        "디콘": "디지털콘텐츠학부",
        "자대": "자연과학대학",
        "인경관": "인문경상관",
        "국지대": "국제지역대학",
        "전언대": "국가전략언어대학",
        "전언": "국가전략언어",
        "국전언": "국가전략언어대학",
        "융소": "융복합소프트웨어전공"
        # 필요시 추가
    }
    for short, full in abbreviation_map.items():
        query = query.replace(short, full)
    return query

# 3. 벡터스토어 불러오기
#pc = Pinecone(api_key=PINECONE_API_KEY)
#index = pc.Index("hufs-academic-chatbot")
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=OPENAI_API_KEY)
# FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(pram_docs, embedding=embeddings)
vectorstore.add_documents(subject_docs)
vectorstore.add_documents(guidebook_docs)

['323fdfad-d957-4698-b094-608acc17f3c2',
 '0ee3d8ea-8e72-49f5-aafa-8df759c116bf',
 'b21ee079-2e22-419c-a70c-b9ef9456b304',
 '124b4354-9c79-4dd4-b509-646856729a28',
 '24ceebee-6de3-4fc6-b6e1-b5048e812cfe',
 'aaa68043-0963-47ce-86b1-dc1248f3a6bd',
 '4b369614-abac-4665-a3d2-5b286a0a4504',
 '8d793754-884d-4fc8-aec2-b8c8e1b44f21',
 'e1eee535-236d-4575-9462-8a8ff2e79ca0',
 '1d2754a8-024a-461f-8c04-d7f70c03a13f',
 'eaf6d6fd-1f3c-4229-b7f4-0e1e571c0a03',
 '8ba6c73f-868c-4b9e-b5de-d2c5cc70c9a1',
 '3c527fa9-5788-4adc-bc1a-45d02cce0acd',
 'd8a1052d-bd58-413f-be25-dcd377da31e1',
 '61659d63-0f78-476b-9027-b3445a6d435c',
 '1f668334-cc77-485c-8fc1-b338c96f9b91',
 '0fc62d6f-6c67-420c-bf11-e76b0bfd2a07',
 'effffb3e-d813-46e2-9a10-14d362714d13',
 'c32ab30a-9822-4db6-98c0-88dd0efc86aa',
 'a50aef38-6510-4752-91be-cc81a2508bd2',
 '547be600-9834-468e-88d7-2fec8e1358a0',
 'e6fda379-4264-4761-8044-262edfc2ffa5',
 '6dca94b5-0068-4b30-8052-b1b0d0819795',
 '63e2eb5f-e8da-49c5-a344-0644e5e60581',
 'e8403096-494b-

In [10]:
# 4. LLM 로딩
llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo", api_key=OPENAI_API_KEY, streaming=True)

# 5. Retriever 구성
#크로스 인코더 리렝커
reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
compressor = CrossEncoderReranker(model=reranker_model, top_n=15) 
# 문맥 압축리트리버
retriever = ContextualCompressionRetriever(                       
    base_compressor=compressor,
    base_retriever=vectorstore.as_retriever(search_kwargs={"k": 30})
)

#retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# 6. 문맥 인식 리트리버
contextualize_prompt = ChatPromptTemplate.from_messages([
    ("system", "이전 대화를 바탕으로 명확한 질문으로 바꾸거나 그대로 사용하세요."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_prompt)
# 순서 재정렬
reordering = LongContextReorder()
#리트리버 체인
my_retriever = (
    {"input": itemgetter("input"), "chat_history": itemgetter("chat_history")} # 이전사용자 질문
    | history_aware_retriever                                                  # 검색기로 넘김
    | RunnableLambda(lambda docs: reordering.transform_documents(docs))        # 문서로 변
)

# 7. QA 프롬프트 체인
qa_system_prompt = """당신은 한국외국어대학교 학사 행정 정보를 제공하는 챗봇입니다.
아래 문서를 참고하여 정확하고 간결하게 질문에 답변하세요.
만약 문서에 정보가 없다면 모른다고 답하세요. 반드시 한국어로 높임말을 사용하여 답변하세요

학생들이 자주 사용하는 학과 및 전공 약어는 다음과 같이 이해하세요:

- 글스산: 글로벌스포츠산업전공
- 바메공 / BME: 바이오메디컬공학전공
- 이중: 이중전공
- 통대: 통번역대학
- 공대: 공과대학
- 일통: 일본어통번역학과
- 영통: 영어통번역학과
- 독통: 독일어통번역학과
- 스통: 스페인어통번역학과
- 마인어: 말레이시아인도네시아통번역학과
- GBT: Global Business&Technology
- 융인: 융합인재학과
- 중통: 중국어통번역학과
- 국금: 국제금융학과
- 이통: 이탈리아어통번역학과
- 태통: 태국어통번역학과
- 정통: 정보통신공학과
- 산공 / 산경공: 산업경영공학과
- 파에: Finance&AI융합학부
- 데융: AI데이터융합학부
- 글자전 / 자전: 글로벌자유전공학부
- 대영: 대학영어
- 데사: 데이터사이언스
- 국리: 국가리더전공
- 세크: 세르비아·크로아티아
- 그불: 그리스·불가리아
- 전물: 전자물리학과
- 생공: 생명공학과
- 디콘: 디지털콘텐츠학부
- 자대: 자연과학대학
- 인경관: 인문경상관
- 국지대: 국제지역대학
- 전언대 / 전언 / 국전언: 국가전략언어대학
- 융소 / 소웨 / swai: 융복합소프트웨어전공

{context}
"""
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])
qa_chain = create_stuff_documents_chain(llm, qa_prompt)

# 8. 전체 RAG 체인 조립
rag_chain = create_retrieval_chain(my_retriever, qa_chain)

# 9. 대화 히스토리 객체
store = {}
session_id = "test_user"
if session_id not in store:
    store[session_id] = ChatMessageHistory()

conversation_chain = RunnableWithMessageHistory(
    rag_chain,
    lambda session_id: store.setdefault(session_id, ChatMessageHistory()),
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

C:\Users\samsung-user\anaconda3\envs\boo_chat\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

C:\Users\samsung-user\anaconda3\envs\boo_chat\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\samsung-user\.cache\huggingface\hub\models--BAAI--bge-reranker-v2-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [11]:
# 10. 테스트 실행
query = "글스산 이중 필수과목에는 뭐가 있어??"
normalized_query = normalize_query(query)

result = conversation_chain.invoke(
    {"input": normalized_query, "chat_history": []},
    config={"configurable": {"session_id": "test-user"}}
)

print("🧠 챗봇 답변:")
print(result["answer"])

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 16991 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [77]:
docs = vectorstore.similarity_search(normalized_query, k=10)
for doc in docs:
    print(doc.metadata)          # 문서 메타정보 (예: source, page 등)
    print(doc.page_content[:100])  # 문서 내용 미리보기

{'producer': 'Adobe PDF Library 16.0', 'creator': 'Adobe InDesign 16.4 (Macintosh)', 'creationdate': '2024-06-21T10:55:17+09:00', 'moddate': '2024-06-21T11:00:00+09:00', 'trapped': '/False', 'source': 'data/major_guide_2025.pdf', 'total_pages': 37, 'page': 30, 'page_label': '31'}
는 수업입니다.
-  스포츠재무관리: 스포츠 산업 내에서 재무적인 측면을 이해하는 데 필
요한 기초적인 개념과 도구들을 다루는 과목입니다.
-  스포츠이벤트기획 및 실습: 스포츠 
{'producer': 'Adobe PDF Library 16.0', 'creator': 'Adobe InDesign 16.4 (Macintosh)', 'creationdate': '2024-06-21T10:55:17+09:00', 'moddate': '2024-06-21T11:00:00+09:00', 'trapped': '/False', 'source': 'data/major_guide_2025.pdf', 'total_pages': 37, 'page': 30, 'page_label': '31'}
입 등의 다양한 영역에서 연구와 교육을 진행하고자 합니다. AI를 활용
한 맞춤형 컨텐츠 제작, 스포츠 이벤트의 디지털화와 새로운 경험 제공, 
그리고 데이터 기반의 마케팅 전략 
{'producer': 'Adobe PDF Library 16.0', 'creator': 'Adobe InDesign 16.4 (Macintosh)', 'creationdate': '2024-06-21T10:55:17+09:00', 'moddate': '2024-06-21T11:00:00+09:00', 'trapped': '/False', 'source': 'data/major_guide_2025.pdf', 'total

In [78]:
# 10. 테스트 실행
query = "스포츠이벤트 수업은 담당 교수님 누구셔?"
normalized_query = normalize_query(query)

result = conversation_chain.invoke(
    {"input": normalized_query, "chat_history": []},
    config={"configurable": {"session_id": "test-user"}}
)

print("🧠 챗봇 답변:")
print(result["answer"])

🧠 챗봇 답변:
죄송합니다, 현재 제가 가지고 있는 정보에는 스포츠이벤트 수업을 담당하는 교수님의 정보가 포함되어 있지 않습니다. 교수님의 정보를 확인하려면 대학의 학사행정부나 학과 사무실에 문의하시는 것이 좋을 것 같습니다.


In [63]:
# 10. 테스트 실행
query = "해당 전공에 스포츠실기 수업도 있어?, 스포츠 실기 종목은 뭐가 있어?"
normalized_query = normalize_query(query)

result = conversation_chain.invoke(
    {"input": normalized_query, "chat_history": []},
    config={"configurable": {"session_id": "test-user"}}
)

print("🧠 챗봇 답변:")
print(result["answer"])

🧠 챗봇 답변:
글로벌스포츠산업전공에서는 매주 목요일마다 학년 별로 다양한 스포츠 종목에 대한 실습을 수업으로 진행합니다. 주로 다음과 같은 스포츠 종목이 포함될 수 있습니다:
- 축구
- 농구
- 야구
- 골프

이를 통해 학생들은 다양한 스포츠 종목에 대한 경험과 이해를 높일 뿐만 아니라 스포츠산업에서의 실무 능력을 키울 수 있습니다.


In [64]:
# 10. 테스트 실행
query = "바메공 졸업하려면 몇학점 들어야돼?"
normalized_query = normalize_query(query)

result = conversation_chain.invoke(
    {"input": normalized_query, "chat_history": []},
    config={"configurable": {"session_id": "test-user"}}
)

print("🧠 챗봇 답변:")
print(result["answer"])

🧠 챗봇 답변:
바이오메디컬공학전공을 졸업하기 위해서는 총 134학점을 이수해야 합니다. 이 학점은 전공 과목, 부전공 과목, 교양 과목 등을 포함한 졸업 요건을 충족시키기 위한 총 이수 학점입니다.


In [79]:
# 10. 테스트 실행
query = "전공 심화 했을 때 각각 몇 학점 들어야되는지 알려줘"
normalized_query = normalize_query(query)

result = conversation_chain.invoke(
    {"input": normalized_query, "chat_history": []},
    config={"configurable": {"session_id": "test-user"}}
)

print("🧠 챗봇 답변:")
print(result["answer"])

🧠 챗봇 답변:
글로벌스포츠산업전공에서 전공 심화를 선택했을 때, 각각의 전공 심화에 필요한 학점은 다음과 같습니다:
- 전공심화(단일전공): 32학점
- 전공심화+부전공: 21학점

따라서, 전공 심화를 선택하면 32학점을 이수해야 하며, 전공 심화와 부전공을 함께 선택하면 21학점을 이수해야 합니다.


In [80]:
docs = vectorstore.similarity_search(normalized_query, k=10)
for doc in docs:
    print(doc.metadata)          # 문서 메타정보 (예: source, page 등)
    print(doc.page_content[:100])  # 문서 내용 미리보기

{'doc_type': '수강편람', 'page': 44, 'num_elements': 10, 'element_ids': [343, 344, 345, 346, 347, 348, 349, 350, 351, 352], 'categories': ['table', 'heading1', 'paragraph', 'footer']}
## [paragraph] ##
2) 2015년 이후 학번 (총 42학점 – 전공기초 18학점 이내, 전공심화 24학점 이상)
[이수학점표]

## [table] ##
| 전공기초
{'doc_type': '수강편람', 'page': 64, 'num_elements': 6, 'element_ids': [522, 523, 524, 525, 526, 527], 'categories': ['heading1', 'paragraph', 'footer', 'list', 'table']}
| 학번별 최소 이수 학점 ※15학번이후는 국제금융학과, GBT학부 영역에서 최소 18학점 이상 / 수학과, 통계학과 영역에서 최소 18학점 이상 이수하면 되지만 총학점은 42학점
{'doc_type': '수강편람', 'page': 100, 'num_elements': 2, 'element_ids': [865, 866], 'categories': ['table', 'footer']}
| 수학과 | 1 | 정수론 | 정수론(2학기) | 7개 영역에서 각 1과목이상, 총 7과목/21학점 이상 이수 |
| 수학과 | 2 | 복소해석학 | 복소변수함수론(1학기) | 
{'doc_type': '수강편람', 'page': 76, 'num_elements': 11, 'element_ids': [623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633], 'categories': ['heading1', 'paragraph', 'footer', 'list', 'table']}
## [list] ##
- ① 매학기 성적 평점평균이 3.75 미만인 경우


In [68]:
# 10. 테스트 실행
query = "글스산 졸업학점 채울 때 자선도 꼭 들어야돼?"
normalized_query = normalize_query(query)

result = conversation_chain.invoke(
    {"input": normalized_query, "chat_history": []},
    config={"configurable": {"session_id": "test-user"}}
)

print("🧠 챗봇 답변:")
print(result["answer"])

🧠 챗봇 답변:
글로벌스포츠산업전공에서 졸업학점을 채울 때 자선 과목을 꼭 이수해야 하는지 여부는 해당 전공의 규정이나 대학의 학사 정책에 따라 다를 수 있습니다. 따라서, 자선 과목을 이수해야 하는지 여부에 대해서는 해당 전공의 학사 담당자나 학사행정부에 문의하시는 것이 가장 확실한 방법일 것입니다.


In [69]:
docs = vectorstore.similarity_search(normalized_query, k=10)
for doc in docs:
    print(doc.metadata)          # 문서 메타정보 (예: source, page 등)
    print(doc.page_content[:100])  # 문서 내용 미리보기

{'producer': 'Adobe PDF Library 16.0', 'creator': 'Adobe InDesign 16.4 (Macintosh)', 'creationdate': '2024-06-21T10:55:17+09:00', 'moddate': '2024-06-21T11:00:00+09:00', 'trapped': '/False', 'source': 'data/major_guide_2025.pdf', 'total_pages': 37, 'page': 31, 'page_label': '32'}
Q3. 스포츠를 직접 하는 수업도 있나요?
A3.  네! 저희는 매주 목요일마다 학년 별로 축구, 농구, 야구, 골프 등의 
스포츠 종목에 대한 실습을 수업으로 진행하고 있습니다.
{'filename': '1학기전공_과목정보(글로벌).csv', '개설영역': '(글로벌) - 글로벌e스포츠매니지먼트전공', 'doc_type': '강의시간표'}
[글로벌e스포츠산업론] 2학년 대상 / 교수: 서형석 / 강의실: 화 4 5 6 (1511) / 학점: 3 / 계획서: 있음

[글로벌e스포츠컨텐츠기획및실습] 3학년 대상 / 교수
{'doc_type': '수강편람', 'page': 22, 'num_elements': 5, 'element_ids': [164, 165, 166, 167, 168], 'categories': ['heading1', 'paragraph', 'footer', 'list', 'table']}
. 다만, 외국인 학생은 본인 국적언어 어학학습 관련 과목 수강은 인정하지 않음. ’자선’을 ‘교양’으로 인정 받고자 하는 학생은 ‘자선’ 학점 취득 후 학사종합지원센터(본인의 1
{'producer': 'Adobe PDF Library 16.0', 'creator': 'Adobe InDesign 16.4 (Macintosh)', 'creationdate': '2024-06-21T10:55:17+09:00', 'moddate': '2024-06-21T11:00:00+0